In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.patches as patches
from matplotlib.collections import PatchCollection
from matplotlib.path import Path
import yaml
import h5py

In [2]:
with open("/content/shifted_layer_3x3_128x128.yaml") as f:
    config = yaml.safe_load(f)

In [3]:
print(config)

{'detector': {'detector geometry': [[0.0, 1.0, 0.0, 2.76, -0.5, 0.5, 0.0, 3.5], [0.0, 1.0, 4.76, 12.76, -0.5, 0.5, 0.0, 3.5], [0.0, 1.0, 14.76, 22.76, -0.5, 0.5, 0.0, 3.5], [0.0, 1.0, 24.76, 32.76, -0.5, 0.5, 0.0, 3.5], [0.0, 1.0, 34.76, 42.76, -0.5, 0.5, 0.0, 3.5], [0.0, 1.0, 44.76, 52.76, -0.5, 0.5, 0.0, 3.5], [0.0, 1.0, 54.76, 62.76, -0.5, 0.5, 0.0, 3.5], [0.0, 1.0, 64.76, 72.76, -0.5, 0.5, 0.0, 3.5], [0.0, 1.0, 74.76, 82.76, -0.5, 0.5, 0.0, 3.5], [0.0, 1.0, 84.76, 92.76, -0.5, 0.5, 0.0, 3.5], [0.0, 1.0, 94.76, 102.76, -0.5, 0.5, 0.0, 3.5], [0.0, 1.0, 104.76, 107.52, -0.5, 0.5, 0.0, 3.5], [30.18, 33.18, 0.18, 3.18, -0.5, 0.5, 1.0, 0.475], [30.18, 33.18, 6.9, 9.9, -0.5, 0.5, 2.0, 0.475], [30.18, 33.18, 13.62, 16.62, -0.5, 0.5, 3.0, 0.475], [30.18, 33.18, 20.34, 23.34, -0.5, 0.5, 4.0, 0.475], [30.18, 33.18, 27.06, 30.06, -0.5, 0.5, 5.0, 0.475], [30.18, 33.18, 33.78, 36.78, -0.5, 0.5, 6.0, 0.475], [30.18, 33.18, 40.5, 43.5, -0.5, 0.5, 7.0, 0.475], [30.18, 33.18, 47.22, 50.22, -0.5, 0.5

In [4]:
from typing import TypedDict

Transform = TypedDict("Transform", {"angle": float, "trans_r": float, "trans_t": float})


def transform_verts(verts: np.ndarray, trans: Transform) -> np.ndarray:
    angle = np.deg2rad(trans["angle"])
    trans_r = trans["trans_r"]
    trans_t = trans["trans_t"]
    mtrans = np.array([[np.cos(angle), -np.sin(angle)], [np.sin(angle), np.cos(angle)]])
    return np.array(
        [np.matmul(mtrans, vert + np.array([trans_r, trans_t])) for vert in verts]
    )


def geom2verts(geom: np.ndarray, trans: Transform) -> np.ndarray:
    verts = np.array(
        [
            [geom[0], geom[2]],
            [geom[1], geom[2]],
            [geom[1], geom[3]],
            [geom[0], geom[3]],
            [geom[0], geom[2]],
        ]
    )
    return transform_verts(verts, trans)


def verts_to_patch(verts: np.ndarray) -> PatchCollection:
    codes = [
        Path.MOVETO,
        Path.LINETO,
        Path.LINETO,
        Path.LINETO,
        Path.CLOSEPOLY,
    ]
    path = Path(verts, codes)
    return patches.PathPatch(path, facecolor="orange", ec="none")


def geoms_to_patchcollection(
    geoms: np.ndarray, trans_list: list[Transform], fc: str = "orange", ec: str = "none"
) -> matplotlib.patches.PathPatch:
    verts_list = []
    for trans in trans_list:
        for geom in geoms:
            verts_list.append(geom2verts(geom, trans))
    return PatchCollection(
        [verts_to_patch(verts) for verts in verts_list], fc=fc, ec=ec
    )


def get_det_geoms(yamlConfig):
    indices = np.asarray(
        yamlConfig["detector"]["active geometry indices"], dtype=np.int32
    )
    active_dets = []
    for idx in indices:
        active_dets.append(geoms[geoms[:, 6] == idx][0])
    return np.array(active_dets)


def get_geom_center_xy(geom):
    return (geom[:4:2] + geom[1:4:2]) * 0.5


'\ndef get_geom_center_xy(geom):\n    # Use only the left edge x-coordinate (geom[0]) instead of averaging with right edge\n    x = geom[0]\n    # Keep the y-coordinate calculation the same (average of top and bottom)\n    y = (geom[2] + geom[3]) * 0.5\n    return np.array([x, y])\n'

In [18]:
#USE THIS CELL BLOCK WHEN PLOTTING FOR NUMPY BASED APPROACH
plt.ioff()
fov_dims = np.array(config["FOV"]["N voxels xyz"]) * np.array(
    config["FOV"]["mm per voxel xyz"]
)
geoms = np.array(config["detector"]["detector geometry"])

trans_t = -(np.max(geoms[:, 3]) + np.min(geoms[:, 2])) / 2
det_dims = np.array(
    [
        np.max(geoms[:, 1]) - np.min(geoms[:, 0]),
        np.max(geoms[:, 3]) - np.min(geoms[:, 2]),
        np.max(geoms[:, 5]) - np.min(geoms[:, 4]),
    ]
)

trans_r = config["relation"]["radial shift"]["data"][0]
trans_list = [
    {
        "angle": config["relation"]["rotation"]["data"][0],
        "trans_r": trans_r,
        "trans_t": trans_t,
    }
]

# patch = verts2rect(transform_verts(verts,trans))
active_det_geoms = geoms[geoms[:, 6] != 0]
plate_geoms = geoms[geoms[:, 6] == 0]

with h5py.File("/content/shifted_layer_3x3_114x114_128fov6rotations_1mmpervoxel (1).hdf5", "r") as f:
    data = f["sysmat"]
    print(data.shape)
    gid = 11
    fov_nvx = np.array(config["FOV"]["N voxels xyz"])[0:2]
    # ppdf = np.empty((fov_nvx))
    det_geoms = get_det_geoms(config)
    fig, ax = plt.subplots(figsize=(15, 10), dpi=300)
    for gid in [0, 10, 25, 67, 120]:
    #for gid in range(16):
        ppdf = data[0, 0, 0, gid].reshape((fov_nvx[0], fov_nvx[1]))

        ax.add_patch(
            plt.Rectangle(
                -fov_dims * 0.5, fov_dims[0], fov_dims[1], fc="none", ec="k", ls="--"
            )
        )
        det_coll = geoms_to_patchcollection(active_det_geoms, trans_list)
        plate_coll = geoms_to_patchcollection(plate_geoms, trans_list, fc="gray")
        coll = geoms_to_patchcollection(geoms, trans_list)
        ax.add_collection(det_coll)
        ax.add_collection(plate_coll)
        # ax.add_collection(coll)
        # ax.set_xlim((trans_r + det_dims[0]) * (-1.1), (trans_r + det_dims[0]) * 1.1)
        # ax.set_ylim((trans_r + det_dims[0]) * (-1.1), (trans_r + det_dims[0]) * 1.1)
        ax.set_xlim((fov_dims[0]) * (-1.1), (trans_r + det_dims[0]) * 1.1)
        ax.set_ylim((det_dims[0]) * (-1.1), (det_dims[0]) * 1.1)
        ax.set_aspect("equal")
        this_geom = np.array([det_geoms[gid]])
        '''
        #OG CODE
        geom_center_xy = transform_verts(
            [get_geom_center_xy(det_geoms[gid])],
            {"angle": 0, "trans_r": trans_r, "trans_t": trans_t},
        )

        '''
        #MY CHANGE
        geom_center_xy = transform_verts(
        [get_geom_center_xy(det_geoms[gid])],
        {"angle": config["relation"]["rotation"]["data"][0], "trans_r": trans_r, "trans_t": trans_t},)


        # print(this_geom)
        this_geom_coll = geoms_to_patchcollection(this_geom, trans_list, fc="r")
        '''
        aperture_y = (plate_geoms[:-1,3]+plate_geoms[1:,2])*0.5+trans_t
        aperture_centers = np.stack(
            (np.full(aperture_y.shape[0], trans_r + 0.5), aperture_y)
        ).T
        '''
        # Calculate aperture_y without adding trans_t directly
        aperture_y = (plate_geoms[:-1, 3] + plate_geoms[1:, 2]) * 0.5
        # Apply transformation to aperture centers after calculating midpoints
        aperture_centers = np.stack(
            (np.full(aperture_y.shape[0], trans_r + 0.5), aperture_y)
        ).T
        # Transform aperture centers using the same transformation logic
        aperture_centers_transformed = transform_verts(aperture_centers, {
            "angle": config["relation"]["rotation"]["data"][0],
            "trans_r": 0,  # No additional radial shift needed here
            "trans_t": trans_t,
        })

        ax.add_collection(this_geom_coll)
        cb = fig.colorbar(
            ax.imshow(
                ppdf,
                extent=(
                    -fov_dims[0] * 0.5,
                    fov_dims[0] * 0.5,
                    -fov_dims[1] * 0.5,
                    fov_dims[1] * 0.5,
                ),
                origin="lower",
            ),
            location="left",
            pad=0.07,
        )
        clipbox = plt.Rectangle(
            (ax.get_xlim()[0], ax.get_ylim()[0]),
            geom_center_xy[0, 0] - ax.get_xlim()[0],
            ax.get_ylim()[1] - ax.get_ylim()[0],
            transform=ax.transData,
        )

        #ax.plot(aperture_centers[:, 0], aperture_centers[:, 1], "o", ms=1)
        ax.plot(aperture_centers_transformed[:, 0], aperture_centers_transformed[:, 1], "o", ms=1)
        for pA in aperture_centers_transformed:
          line = ax.axline(geom_center_xy[0], pA, ls="--", lw=0.5)
          line.set_clip_path(clipbox)

        #for pA in aperture_centers:
        #    line = ax.axline(geom_center_xy[0], pA, ls="--", lw=0.5)
        #    line.set_clip_path(clipbox)
        ax.set_xlabel("Transverse plane x (mm)",fontsize=18)
        ax.set_ylabel("Transverse plane y (mm)",fontsize=18)
        fig.tight_layout()
        ax.set_title("detector %03d" % gid)
        fig.savefig("detector %03d" % gid)
        cb.remove()
        ax.cla()
        del det_coll, plate_coll, coll
    del fig, ax


FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = '/content/shifted_layer_3x3_114x114_128fov6rotations_1mmpervoxel (1).hdf5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
#USE THIS BLOCK WHEN PLOTTING FOR TORCH BASED APPROACH
plt.ioff()
fov_dims = np.array(config["FOV"]["N voxels xyz"]) * np.array(
    config["FOV"]["mm per voxel xyz"]
)
geoms = np.array(config["detector"]["detector geometry"])

trans_t = -(np.max(geoms[:, 3]) + np.min(geoms[:, 2])) / 2
det_dims = np.array(
    [
        np.max(geoms[:, 1]) - np.min(geoms[:, 0]),
        np.max(geoms[:, 3]) - np.min(geoms[:, 2]),
        np.max(geoms[:, 5]) - np.min(geoms[:, 4]),
    ]
)

trans_r = config["relation"]["radial shift"]["data"][0]
trans_list = [
    {
        "angle": config["relation"]["rotation"]["data"][0],
        "trans_r": trans_r,
        "trans_t": trans_t,
    }
]

# patch = verts2rect(transform_verts(verts,trans))
active_det_geoms = geoms[geoms[:, 6] != 0]
plate_geoms = geoms[geoms[:, 6] == 0]
with h5py.File("/content/resized_output_1mmpervoxel.hdf5", "r") as f:
    data = f["sysmat"]
    fov_nvx = np.array(config["FOV"]["N voxels xyz"])[0:2]

    det_geoms = get_det_geoms(config)
    fig, ax = plt.subplots(figsize=(15, 10), dpi=300)

    # Loop over projections instead of single detector indices
    for gid in range(10):  # 'a' is number of projections

        # Extract the system matrix projection slice
        ppdf = data[gid].reshape((fov_nvx[0], fov_nvx[1]))

        ax.add_patch(
            plt.Rectangle(
                -fov_dims * 0.5, fov_dims[0], fov_dims[1], fc="none", ec="k", ls="--"
            )
        )

        det_coll = geoms_to_patchcollection(active_det_geoms, trans_list)
        plate_coll = geoms_to_patchcollection(plate_geoms, trans_list, fc="gray")
        coll = geoms_to_patchcollection(geoms, trans_list)

        ax.add_collection(det_coll)
        ax.add_collection(plate_coll)

        ax.set_xlim((fov_dims[0]) * (-1.1), (trans_r + det_dims[0]) * 1.1)
        ax.set_ylim((det_dims[0]) * (-1.1), (det_dims[0]) * 1.1)
        ax.set_aspect("equal")

        this_geom = np.array([det_geoms[gid]])

        geom_center_xy = transform_verts(
            [get_geom_center_xy(det_geoms[gid])],
            {"angle": 0, "trans_r": trans_r, "trans_t": trans_t},
        )

        '''
        geom_center_xy = transform_verts(
            [get_geom_center_xy(det_geoms[gid])],
            {"angle": config["relation"]["rotation"]["data"][0], "trans_r": trans_r, "trans_t": trans_t},
        )
        '''
        this_geom_coll = geoms_to_patchcollection(this_geom, trans_list, fc="r")

        '''
        aperture_y = (plate_geoms[:-1,3] + plate_geoms[1:,2]) * 0.5 + trans_t
        aperture_centers = np.stack(
            (np.full(aperture_y.shape[0], trans_r + 0.5), aperture_y)
        ).T
        '''
        # Calculate aperture_y without adding trans_t directly
        aperture_y = (plate_geoms[:-1, 3] + plate_geoms[1:, 2]) * 0.5
        # Apply transformation to aperture centers after calculating midpoints
        aperture_centers = np.stack(
            (np.full(aperture_y.shape[0], trans_r + 0.5), aperture_y)
        ).T
        # Transform aperture centers using the same transformation logic
        aperture_centers_transformed = transform_verts(aperture_centers, {
            "angle": config["relation"]["rotation"]["data"][0],
            "trans_r": 0,  # No additional radial shift needed here
            "trans_t": trans_t,
        })

        ax.add_collection(this_geom_coll)

        cb = fig.colorbar(
            ax.imshow(
                ppdf,
                extent=(-fov_dims[0] * 0.5, fov_dims[0] * 0.5, -fov_dims[1] * 0.5, fov_dims[1] * 0.5),
                origin="lower",
            ),
            location="left",
            pad=0.07,
        )

        clipbox = plt.Rectangle(
            (ax.get_xlim()[0], ax.get_ylim()[0]),
            geom_center_xy[0, 0] - ax.get_xlim()[0],
            ax.get_ylim()[1] - ax.get_ylim()[0],
            transform=ax.transData,
        )

        '''
        ax.plot(aperture_centers[:, 0], aperture_centers[:, 1], "o", ms=1)
        for pA in aperture_centers:
            line = ax.axline(geom_center_xy[0], pA, ls="--", lw=0.5)
            line.set_clip_path(clipbox)
        '''
        ax.plot(aperture_centers_transformed[:, 0], aperture_centers_transformed[:, 1], "o", ms=1)
        for pA in aperture_centers_transformed:
          line = ax.axline(geom_center_xy[0], pA, ls="--", lw=0.5)
          line.set_clip_path(clipbox)
        ax.set_xlabel("Transverse plane x (mm)", fontsize=18)
        ax.set_ylabel("Transverse plane y (mm)", fontsize=18)
        fig.tight_layout()
        ax.set_title(f"Projection {gid}")
        fig.savefig(f"projection_{gid}.png")

        cb.remove()
        ax.cla()
        del det_coll, plate_coll, coll

    del fig, ax


In [ ]:
import h5py
import numpy as np

def resize_hdf5(input_file, output_file, dataset_name):
    """
    Resize an HDF5 dataset from (1,1,6,144,16384) to (864,128,128)

    Parameters:
    input_file (str): Path to input HDF5 file
    output_file (str): Path to output HDF5 file
    dataset_name (str): Name of the dataset to resize
    """
    with h5py.File(input_file, 'r') as f_in:
        data = f_in[dataset_name][:]  # Read the entire dataset

        # Verify input dimensions
        if data.shape != (1, 1, 6, 144, 16384):
            raise ValueError(f"Expected shape (1,1,6,144,16384), got {data.shape}")

        # Reshape the data
        # First flatten the first three dimensions (1*1*6 = 6)
        # Then reshape 144*16384 into 144*128*128
        data = data.reshape(6, 144, 16384)

        # Reorganize to get 864 samples (6*144 = 864)
        data = data.reshape(864, 16384)

        # Finally reshape the last dimension into 128x128
        data = data.reshape(864, 128, 128)

        # Create new file and dataset
        with h5py.File(output_file, 'w') as f_out:
            f_out.create_dataset(dataset_name, data=data, dtype=data.dtype)

# Example usage
if __name__ == "__main__":
    input_file = "/content/shifted_layer_3x3_114x114_128fov6rotations_1mmpervoxel (1).hdf5"
    output_file = "resized_output_1mmpervoxel.hdf5"
    dataset_name = "sysmat"  # Replace with your actual dataset name

    resize_hdf5(input_file, output_file, dataset_name)

In [ ]:
######################

In [13]:
import numpy as np
import matplotlib.pyplot as plt
import h5py

plt.ioff()

fov_dims = np.array(config["FOV"]["N voxels xyz"]) * np.array(
    config["FOV"]["mm per voxel xyz"]
)
geoms = np.array(config["detector"]["detector geometry"])

trans_t = -(np.max(geoms[:, 3]) + np.min(geoms[:, 2])) / 2
det_dims = np.array(
    [
        np.max(geoms[:, 1]) - np.min(geoms[:, 0]),
        np.max(geoms[:, 3]) - np.min(geoms[:, 2]),
        np.max(geoms[:, 5]) - np.min(geoms[:, 4]),
    ]
)

trans_r = config["relation"]["radial shift"]["data"][0]
trans_list = [
    {
        "angle": config["relation"]["rotation"]["data"][0],
        "trans_r": trans_r,
        "trans_t": trans_t,
    }
]

active_det_geoms = geoms[geoms[:, 6] != 0]
plate_geoms = geoms[geoms[:, 6] == 0]

with h5py.File("/content/system_matrix_000.hdf5", "r") as f:
    data = f["system matrix"]  # New dataset name
    print(data.shape)  # Should print (864, fov_n_voxels_xyz[0], fov_n_voxels_xyz[1])

    fov_nvx = np.array(config["FOV"]["N voxels xyz"])[:2]
    det_geoms = get_det_geoms(config)
    fig, ax = plt.subplots(figsize=(15, 10), dpi=300)

    # Iterate over detector cuboids
    for gid in range(17):  # Change range if needed
        ppdf = data[gid].reshape((fov_nvx[1], fov_nvx[0]))

        ax.add_patch(
            plt.Rectangle(
                -fov_dims * 0.5, fov_dims[0], fov_dims[1], fc="none", ec="k", ls="--"
            )
        )
        det_coll = geoms_to_patchcollection(active_det_geoms, trans_list)
        plate_coll = geoms_to_patchcollection(plate_geoms, trans_list, fc="gray")
        coll = geoms_to_patchcollection(geoms, trans_list)
        ax.add_collection(det_coll)
        ax.add_collection(plate_coll)

        ax.set_xlim((fov_dims[0]) * (-1.1), (trans_r + det_dims[0]) * 1.1)
        ax.set_ylim((det_dims[0]) * (-1.1), (det_dims[0]) * 1.1)
        ax.set_aspect("equal")

        this_geom = np.array([det_geoms[gid]])

        '''
        geom_center_xy = transform_verts(
            [get_geom_center_xy(det_geoms[gid])],
            {"angle": 0, "trans_r": trans_r, "trans_t": trans_t},
        )

        '''
        geom_center_xy = transform_verts(
            [get_geom_center_xy(det_geoms[gid])],
            {"angle": config["relation"]["rotation"]["data"][0], "trans_r": trans_r, "trans_t": trans_t},
        )


        this_geom_coll = geoms_to_patchcollection(this_geom, trans_list, fc="r")
        aperture_y = (plate_geoms[:-1, 3] + plate_geoms[1:, 2]) * 0.5 + trans_t
        aperture_centers = np.stack(
            (np.full(aperture_y.shape[0], trans_r + 0.5), aperture_y)
        ).T
        '''

        # Calculate aperture_y without adding trans_t directly
        aperture_y = (plate_geoms[:-1, 3] + plate_geoms[1:, 2]) * 0.5
        # Apply transformation to aperture centers after calculating midpoints
        aperture_centers = np.stack(
            (np.full(aperture_y.shape[0], trans_r + 0.5), aperture_y)
        ).T
        # Transform aperture centers using the same transformation logic
        aperture_centers_transformed = transform_verts(aperture_centers, {
            "angle": config["relation"]["rotation"]["data"][0],
            "trans_r": 0,  # No additional radial shift needed here
            "trans_t": trans_t,
        })
        '''
        ax.add_collection(this_geom_coll)

        cb = fig.colorbar(
            ax.imshow(
                ppdf.T,
                extent=(-fov_dims[0] * 0.5, fov_dims[0] * 0.5, -fov_dims[1] * 0.5, fov_dims[1] * 0.5),
                origin="lower",
            ),
            location="left",
            pad=0.07,
        )

        clipbox = plt.Rectangle(
            (ax.get_xlim()[0], ax.get_ylim()[0]),
            geom_center_xy[0, 0] - ax.get_xlim()[0],
            ax.get_ylim()[1] - ax.get_ylim()[0],
            transform=ax.transData,
        )


        ax.plot(aperture_centers[:, 0], aperture_centers[:, 1], "o", ms=1)
        for pA in aperture_centers:
            line = ax.axline(geom_center_xy[0], pA, ls="--", lw=0.5)
            line.set_clip_path(clipbox)
        '''
        ax.plot(aperture_centers_transformed[:, 0], aperture_centers_transformed[:, 1], "o", ms=1)
        for pA in aperture_centers_transformed:
          line = ax.axline(geom_center_xy[0], pA, ls="--", lw=0.5)
          line.set_clip_path(clipbox)
        '''
        ax.set_xlabel("Transverse plane x (mm)", fontsize=18)
        ax.set_ylabel("Transverse plane y (mm)", fontsize=18)
        fig.tight_layout()
        ax.set_title("detector %03d" % gid)
        fig.savefig("detector %03d" % gid)
        cb.remove()
        ax.cla()
        del det_coll, plate_coll, coll

    del fig, ax

(864, 128, 128)
